In [6]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
from PIL import Image

In [3]:
# Functions
def importImages(path):
    "Function to import all images from a folder"
    images = []
    for filename in os.listdir(path):
        img = cv.imread(os.path.join(path, filename))
        if img is not None:
            images.append(img)
    return images

def showImage(preprocessed=[False,], number=1):
    "Function to show an image"
    if preprocessed[0]:
        cv.imshow("Image",preprocessed[1])
    else:
        cv.imshow("Image", importImages("data/easy")[number])
    cv.waitKey(0)
    cv.destroyAllWindows()

def preprocess(gamma=[False, 1.5], equalize=False, grayscale=False, bilateralFilter=False, gaussianBlur=False):
    "Function to preprocess the images"
    images = importImages("data/easy")
    preprocessed = []
    for img in images:
        # Convert to grayscale if needed
        if grayscale:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
        # Apply bilateral filter if needed
        if bilateralFilter:
            img = cv.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
        
        # Apply Gaussian blur if needed
        if gaussianBlur:
            img = cv.GaussianBlur(img, (9, 9), 2)
        
        # Gamma correction
        if gamma[0]:
            img = img / 255.0  # Normalize to 0-1
            img = np.power(img, gamma[1])
            img = (img * 255).clip(0, 255).astype(np.uint8)  # Denormalize to 0-255
        
        # Equalize histogram
        if equalize:
            if len(img.shape) == 2:  # Grayscale image
                img = cv.equalizeHist(img)
            else:  # Color image
                img_yuv = cv.cvtColor(img, cv.COLOR_BGR2YUV)
                img_yuv[:, :, 0] = cv.equalizeHist(img_yuv[:, :, 0])
                img = cv.cvtColor(img_yuv, cv.COLOR_YUV2BGR)
        
        preprocessed.append(img)
    return preprocessed

Gaussian blur reduce the errors on edges, so it should reduce false positive, focus on significative changes. Bilateral add a sector to preserve edges.

In [33]:
showImage()

In [10]:
preprocessed = preprocess(gamma=[True, 0.7], equalize=False, grayscale=False, bilateral_filter=True)

In [11]:
showImage(preprocessed=[True, preprocessed[0]])

: 

In [1]:
"""
def detectRoadSigns(image_path):
    # Carica l'immagine
    img = cv.imread(image_path)
    if img is None:
        print(f"Error: Cannot load image at {image_path}")
        return
    
    # Preprocess
    preprocessed = preprocess(gamma=[False, 0.7], equalize=False, grayscale=True, gaussianBlur=True)

    # Utilizza il rilevamento dei bordi di Canny
    edges = cv.Canny(preprocessed[0], 100, 200)
    
    # Applica la trasformata di Hough per rilevare cerchi
    circles = cv.HoughCircles(
    preprocessed[0],
    cv.HOUGH_GRADIENT,
    dp=1.2,
    minDist=50,
    param1=100,
    param2=40,
    minRadius=15,
    maxRadius=50
)
    
    # Se vengono rilevati cerchi, disegnali sull'immagine originale
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv.circle(img, (x, y), r, (0, 255, 0), 4)
            cv.rectangle(img, (x - r, y - r), (x + r, y + r), (0, 128, 255), 2)
    
    return img

showImage([True, detectRoadSigns("data/easy/rs3.jpg")])
"""


'\ndef detectRoadSigns(image_path):\n    # Carica l\'immagine\n    img = cv.imread(image_path)\n    if img is None:\n        print(f"Error: Cannot load image at {image_path}")\n        return\n    \n    # Preprocess\n    preprocessed = preprocess(gamma=[False, 0.7], equalize=False, grayscale=True, gaussianBlur=True)\n\n    # Utilizza il rilevamento dei bordi di Canny\n    edges = cv.Canny(preprocessed[0], 100, 200)\n    \n    # Applica la trasformata di Hough per rilevare cerchi\n    circles = cv.HoughCircles(\n    preprocessed[0],\n    cv.HOUGH_GRADIENT,\n    dp=1.2,\n    minDist=50,\n    param1=100,\n    param2=40,\n    minRadius=15,\n    maxRadius=50\n)\n    \n    # Se vengono rilevati cerchi, disegnali sull\'immagine originale\n    if circles is not None:\n        circles = np.round(circles[0, :]).astype("int")\n        for (x, y, r) in circles:\n            cv.circle(img, (x, y), r, (0, 255, 0), 4)\n            cv.rectangle(img, (x - r, y - r), (x + r, y + r), (0, 128, 255), 2)\n 